In [ ]:
%run 05_generation.ipynb

In [5]:
import redis

redis_client = redis.Redis(
    host="localhost",
    port=6379,
    db=0,
    decode_responses=True  # return strings instead of bytes
)

To Run Redis using Docker -->  
Open Docker in your device and the run  
```bash
docker run -d --name redis_server -p 6379:6379 redis
```
To Check If its running 
```bash
docker ps
```

In [6]:
import re

def normalize_query(query: str) -> str:
    q = query.lower().strip()
    q = re.sub(r"\s+", " ", q)  # collapse multiple spaces
    return q


In [8]:
def cache_get(query: str):
    key = normalize_query(query)
    return redis_client.get(key)

In [9]:
def cache_set(query: str, answer: str, ttl=86400):  # 24 hours
    key = normalize_query(query)
    redis_client.set(key, answer, ex=ttl)

In [1]:
def answer_query(query):

    # STEP 0 — ROUTER
    decision = route_query(query)
    print("Router decision:", decision)

    if decision == "direct":
        answer = generator_llm.invoke(query).content
        cache_set(query, answer)
        return answer

    # STEP 1 — CACHE CHECK
    cached = cache_get(query)
    if cached:
        print("CACHE HIT")
        return cached

    print("CACHE MISS → Running full RAG pipeline...")

    # STEP 2 — RETRIEVE
    retrieved = retrieve_candidates(query)

    # STEP 3 — RERANK
    reranked = rerank_documents(query, retrieved, top_k=6)

    # STEP 4 — GENERATE
    answer = generate_answer(query, reranked)

    # STEP 5 — STORE IN CACHE
    cache_set(query, answer)

    return answer

In [15]:
answer_query("What did Microsoft report about cloud revenue in 2024?")


CACHE MISS → Running full RAG pipeline...
Router decision: rag
BM25 retrieved: 10 chunks
Semantic retrieved: 10 chunks
Combined unique: 17


'Microsoft reported that its Microsoft Cloud revenue increased 23% to $137.4 billion in fiscal year 2024.'

In [18]:
answer_query("What did Microsoft report about cloud revenue in 2024?")

CACHE HIT


'Microsoft reported that its Microsoft Cloud revenue increased 23% to $137.4 billion in fiscal year 2024.'